# **Data Collection**


Dataset available given link -
https://drive.google.com/file/d/1ITbDvhLwyTTkuUYfNjOKhcIZh7hDgi64/view?usp=sharing

In [1]:
pwd

'/home/wsuser/work'

In [2]:
!pip install tensorflow==2.7.1

     |████████████████████████████████| 495.2 MB 19 kB/s /s eta 0:00:01█████                           | 76.2 MB 92.4 MB/s eta 0:00:05             | 134.9 MB 110.3 MB/s eta 0:00:04             | 139.7 MB 110.3 MB/s eta 0:00:04    |██████████                      | 155.4 MB 110.3 MB/s eta 0:00:040.3 MB/s eta 0:00:03██▎                  | 206.3 MB 110.3 MB/s eta 0:00:03�█████████▏                 | 219.6 MB 85.6 MB/s eta 0:00:04�█████████▊                 | 228.5 MB 85.6 MB/s eta 0:00:04/s eta 0:00:04/s eta 0:00:03MB/s eta 0:00:13��█▎            | 298.0 MB 15.5 MB/s eta 0:00:13██████████████▌           | 317.4 MB 15.5 MB/s eta 0:00:12██████████████▊           | 319.9 MB 15.5 MB/s eta 0:00:12:00:11:00:11:00:10███▏        | 358.5 MB 98.4 MB/s eta 0:00:02�██████████▎        | 360.4 MB 98.4 MB/s eta 0:00:02██████████████████████▌       | 378.6 MB 98.4 MB/s eta 0:00:02��███████▌     | 409.3 MB 113.6 MB/s eta 0:00:01��███████▋     | 412.0 MB 113.6 MB/s eta 0:00:01��██████████████████▊    | 428

In [3]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6urDOmS-BHxjQeXtydrlLChJz-fRW5JKOm_yJOObiQXh',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'realtimecommunicationsystempowere-donotdelete-pr-kzqyqinjnvqefn'
object_key = 'Dataset.zip'

streaming_body_2 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [4]:
# Unzip the Dataset Zip File
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_2.read()), 'r')
file_paths = unzip.namelist()
for path in file_paths:
    unzip.extract(path)

In [5]:
ls Dataset

test_set/  training_set/


In [6]:
from keras.preprocessing.image import ImageDataGenerator

2022-11-08 04:16:21.601625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow


In [7]:
train_datagen = ImageDataGenerator(rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    vertical_flip=True,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

In [8]:
train_set = train_datagen.flow_from_directory(
    '/home/wsuser/work/Dataset/training_set',
    batch_size = 100,
    class_mode = 'categorical',
    target_size = (64,64)
)

test_set = test_datagen.flow_from_directory(
    '/home/wsuser/work/Dataset/test_set',
    batch_size = 50,
    class_mode = 'categorical',
    target_size = (64,64)
)

Found 15750 images belonging to 9 classes.
Found 2250 images belonging to 9 classes.


# **Model Building**


**Importing the Libraries**

In [9]:
from keras.models import Sequential
from tensorflow import keras
from keras import optimizers

**Adding the layers**

In [10]:
classifier = keras.Sequential([
    keras.layers.Conv2D(16,(3,3), activation='relu', input_shape=(64,64,3)),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(32,(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Conv2D(64,(3,3), activation='relu'),
    keras.layers.MaxPool2D((2,2)),
    keras.layers.Flatten(),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(9, activation='softmax')
])

2022-11-08 04:31:03.268870: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-11-08 04:31:03.268918: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)


**Compiling the Model**

In [11]:
classifier.compile(
    optimizer = 'adam',
    loss = 'categorical_crossentropy',
    metrics =['accuracy']
)
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 64)         0

**Fitting and Saving the Model**

In [12]:
# Model is fit
model = classifier.fit(
    train_set,
    steps_per_epoch=len(train_set),
    epochs=10,
    validation_data = test_set,
    validation_steps = len(test_set)
)

Epoch 1/10
158/158 [==============================] - 73s 458ms/step - loss: 1.2650 - accuracy: 0.5194 - val_loss: 0.2095 - val_accuracy: 0.9956
Epoch 2/10
158/158 [==============================] - 72s 453ms/step - loss: 0.5730 - accuracy: 0.7848 - val_loss: 0.2497 - val_accuracy: 0.9231
Epoch 3/10
158/158 [==============================] - 74s 466ms/step - loss: 0.3884 - accuracy: 0.8547 - val_loss: 0.0836 - val_accuracy: 0.9929
Epoch 4/10
158/158 [==============================] - 72s 451ms/step - loss: 0.3074 - accuracy: 0.8877 - val_loss: 0.0642 - val_accuracy: 0.9960
Epoch 5/10
158/158 [==============================] - 72s 453ms/step - loss: 0.2722 - accuracy: 0.9010 - val_loss: 0.0364 - val_accuracy: 0.9987
Epoch 6/10
158/158 [==============================] - 74s 465ms/step - loss: 0.2254 - accuracy: 0.9197 - val_loss: 0.0340 - val_accuracy: 0.9960
Epoch 7/10
158/158 [==============================] - 73s 459ms/step - loss: 0.2111 - accuracy: 0.9256 - val_loss: 0.0501 - val_ac

In [13]:
classifier.save('IBM_Model.h5') # Model is saved

In [14]:
!tar -zcvf IBM_TrainedModel.tgz IBM_Model.h5

IBM_Model.h5


In [15]:
ls -1

Dataset/
IBM_Model.h5
IBM_TrainedModel.tgz


In [16]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 12.2 MB/s eta 0:00:01


In [17]:
from ibm_watson_machine_learning import APIClient
wml_credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": "8KZME-Ln5R_HH4ffa2LGOzc2g_TyRV_PtkVBs2jk5Q2p"
}

client = APIClient(wml_credentials)

In [18]:
def guid_from_space_name(client, space_name):
    space = client.spaces.get_details()
    return (next(item for item in space['resources'] if item['entity']["name"] == space_name)['metadata']['id'])

In [19]:
space_uid = guid_from_space_name(client, 'model')
print("Space UID : ", space_uid)

Space UID :  3b6ccea1-7b7e-4385-8c14-ba35b7760a2e


In [20]:
client.set.default_space(space_uid)

'SUCCESS'

In [22]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
runtime-22.1-py3.9             12b83a1

In [23]:
software_spec_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")
software_spec_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [24]:
model_details = client.repository.store_model(model='IBM_TrainedModel.tgz', meta_props= {
    client.repository.ModelMetaNames.NAME : 'CNN',
    client.repository.ModelMetaNames.TYPE : "tensorflow_2.7",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : software_spec_uid
})

model_id = client.repository.get_model_uid(model_details)

This method is deprecated, please use get_model_id()


/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/ibm_watson_machine_learning/repository.py:1453: UserWarning: This method is deprecated, please use get_model_id()
  warn("This method is deprecated, please use get_model_id()")


In [25]:
model_id

'e19515ea-424d-4888-81ea-77db8b32e01c'

# Testing the Model

In [41]:
import PIL
from keras.models import load_model
from keras.preprocessing import image

In [42]:
trained_model = load_model('IBM_Model.h5')

In [51]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='6urDOmS-BHxjQeXtydrlLChJz-fRW5JKOm_yJOObiQXh',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'realtimecommunicationsystempowere-donotdelete-pr-kzqyqinjnvqefn'
object_key = 'Test_Image(G).jpeg'

streaming_body_5 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [52]:
img = image.load_img(streaming_body_5, target_size=(64,64))

TypeError: expected str, bytes or os.PathLike object, not StreamingBody